## IRC Technical Assessment
I chose to use PostgreSQL to complete this assessment.

In [3]:
import psycopg2


In [4]:
conn = psycopg2.connect(dbname="irc", user="brianrennie", password="brianrennie")

cursor = conn.cursor()

In [5]:

cursor.execute("SELECT * FROM accounts limit 5;")
accounts = cursor.fetchall()
accounts

[('0x387D3247D2A5536C1C666CFBB259659F', 'household', 'United States'),
 ('0xB9C58CE7117BC12ED7DDEF301F798AA7', 'household', 'United States'),
 ('0xB0A6B44F19054E3BBF96FAA980267AE5', 'household', 'United States'),
 ('0x312EF920F30C85CEA85D1946B33AF12F', 'household', 'United States'),
 ('0x8A1F32025B28D0B32E84CF37CE392E18', 'household', 'United States')]

In [6]:
cursor.execute("SELECT close_date FROM donations limit 1;")
donations = cursor.fetchall()
donations

[(datetime.date(2022, 10, 15),)]

## 1. What are the top five organizations by revenue in 2021?

In [7]:
conn = psycopg2.connect(dbname="irc", user="brianrennie", password="brianrennie")

cursor = conn.cursor()

cursor.execute('''SELECT account_id,close_date, SUM(amount) as total_amount
                    FROM donations 
                    WHERE account_id in (
                                         SELECT account_id
                                             FROM accounts
                                             WHERE account_type = 'organization')
                    
                    GROUP BY 1,2
                    HAVING EXTRACT(YEAR FROM close_date) = 2021  
                    ORDER BY total_amount DESC limit 5;''')
result = cursor.fetchall()
result

[('0x6CF1B1B8F8CCB4FA7E3BF638400C6694',
  datetime.date(2021, 3, 9),
  '$25,000.64'),
 ('0xC1638AC40976514922330766B0CDA0E1',
  datetime.date(2021, 8, 25),
  '$24,793.60'),
 ('0x6CF1B1B8F8CCB4FA7E3BF638400C6694',
  datetime.date(2021, 6, 15),
  '$15,747.58'),
 ('0xAB7B829B9A1721326A19A7131A2659CC',
  datetime.date(2021, 4, 20),
  '$5,150.00'),
 ('0xD34FE50ED1B96D1C9D2308CD86F13530',
  datetime.date(2021, 12, 22),
  '$2,000.00')]

## 2. Create an account-level table with aggregate measures of revenue in past week, past month, past year, and all time.

In [8]:
conn = psycopg2.connect(dbname="irc", user="brianrennie", password="brianrennie")

cursor = conn.cursor()

cursor.execute('''WITH last_week_revenue as
                  (
                  SELECT account_id, DATE_PART('week', close_date), SUM(amount)  as last_week_revenue
                    FROM donations 
                    GROUP BY 1,2 
                    HAVING DATE_PART('week', close_date) = 41.0
                    ), 

                  last_year_revenue as
                   (
                   SELECT account_id, DATE_PART('year', close_date), SUM(amount) as last_year_revenue
                    FROM donations 
                    GROUP BY 1,2 
                    HAVING DATE_PART('year', close_date) = 2021
                    ),

                   last_month_revenue as
                   (
                    SELECT account_id, DATE_PART('month', close_date), SUM(amount) as last_month_revenue 
                    FROM donations 
                    GROUP BY 1,2 
                    HAVING DATE_PART('month', close_date) = 9
                    ),

                    all_time_revenue as
                     (
                    SELECT account_id, SUM(amount) as all_time_revenue 
                    FROM donations 
                    GROUP BY 1 
                    )


                   SELECT 
                        lwr.account_id,
                        lwr.last_week_revenue,
                        atr.all_time_revenue,
                        lmr.last_month_revenue,
                        lyr.last_year_revenue
                   FROM all_time_revenue atr
                   JOIN last_week_revenue lwr
                   ON lwr.account_id = atr.account_id
                   JOIN last_month_revenue lmr
                   ON lmr.account_id = lwr.account_id
                   JOIN last_year_revenue lyr
                   ON lyr.account_id = lmr.account_id''')
result = cursor.fetchall()
result

[('0x6EA33453C967D7CF47A136E1C2A810AF',
  '$40.00',
  '$57.40',
  '$10.00',
  '$40.00'),
 ('0x0A3EF6B9B029C5E5B152B6C8EA34D370',
  '$25.00',
  '$1,207.41',
  '$100.00',
  '$183.00'),
 ('0x5D69AC59187D4DA14B6AF2BF932EA5BC',
  '$35.00',
  '$1,599.00',
  '$100.00',
  '$340.20'),
 ('0xD525444F870719C82256CD8076C700D3',
  '$40.00',
  '$761.93',
  '$20.00',
  '$10.00'),
 ('0x4F6AA4D1EA29E02705A5218A4C3FD05F',
  '$45.00',
  '$213.33',
  '$30.00',
  '$45.00'),
 ('0xB3D9C3CD609B4E4A16E81824BBF0C180',
  '$11.00',
  '$398.66',
  '$33.33',
  '$11.00'),
 ('0x637CCB1EEC8B4A1848BAF4A076368672',
  '$19.00',
  '$61.29',
  '$10.30',
  '$19.00'),
 ('0x8B630035A1C147CFD9F8E409E959DB7B',
  '$30.00',
  '$149.01',
  '$12.00',
  '$30.00'),
 ('0xD34FE50ED1B96D1C9D2308CD86F13530',
  '$300.90',
  '$14,785.89',
  '$180.00',
  '$2,690.99'),
 ('0x52576815EA8A926EBE54DF1CF0485148',
  '$50.00',
  '$3,317.73',
  '$85.00',
  '$1,713.47'),
 ('0x752B95389305BA0546668F1D5AD9AF7B',
  '$107.32',
  '$5,061.11',
  '$309.33',


## 3. What is the average first donation of new donors for each billing country?

In [81]:
conn = psycopg2.connect(dbname="irc", user="brianrennie", password="brianrennie")

cursor = conn.cursor()

cursor.execute('''with numbered_donations as
                (
                SELECT a.account_id as account,
                       billing_country,
                       close_date,
                       amount::numeric as amount,
                       ROW_NUMBER () OVER (PARTITION BY a.account_id ORDER BY close_date) as donation_number
                       FROM accounts a
                       JOIN donations d 
                       ON a.account_id = d.account_id
                       
                ),

                first_donations as 
                (
                SELECT account,
                       billing_country,
                       close_date,
                       amount,
                       donation_number
                FROM numbered_donations
                WHERE donation_number = 1
                )     
                
                
                SELECT billing_country, AVG(ROUND(amount,2)) from first_donations GROUP BY 1''')
result = cursor.fetchall()
result

[('Germany', Decimal('36.7918181818181818')),
 ('United Kingdom', Decimal('37.3312500000000000')),
 ('United States', Decimal('479.8499349710982659'))]